In [1]:
#7.1 Writing Functions That Accept Any Number of Arguments

def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

avg(12, 34, 56, 78)

45.0

In [3]:
import html

def make_element(name, value, **attrs):
    keyvals = ['%s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
                  name=name,
                  attrs=attr_str,
                  value=html.escape(value)
    )
    return element

make_element('player', 'Robert Lewandowski', age='33', rating='93' )

'<playerage="33"rating="93">Robert Lewandowski</player>'

In [4]:
#7.2 Writing Functions That Only Accept Keyword Arguments

def maximum(*values, clip=None):
    m = max(values)
    if clip is not None:
        m= clip if clip < m else m
    return m

maximum(65, 199, 45, 98, 17)

199

In [11]:
#7.3 Attaching Informational Metadata to Function Arguments

def avg(first:int, *rest:int) -> int:
    return (first + sum(rest)) / (1 + len(rest))

help(avg)

Help on function avg in module __main__:

avg(first: int, *rest: int) -> int



In [10]:
avg.__annotations__

{}

In [16]:
#7.4 Returning Multiple Values from a Function

def myplayers():
    return 'Derek Carr', 'Hunter Renfrow', 'Darren Waller', 'Davante Adams'

In [19]:
a, b, c, d = myplayers()
print(a)

Derek Carr


In [21]:
#7.5 Defining Functions with Default Arguments

no_value = object()

def add(a, b=no_value):
    if b is no_value:
        return a
    else:
        return(a + b)

In [22]:
add(5)

5

In [23]:
add(5,7)

12

In [7]:
def multiply(a, b=None):
    if b is None:
        return a
    else:
        return(a * b)

multiply(5,6)

30

In [8]:
multiply(5)

5

In [10]:
#7.6 Defining Anonymous or Inline Functions

add = lambda a, b: a + b
add(11,45)

56

In [15]:
grocery_list = [
    'Apple',
    'Turkey',
    'Lettuce',
    'Tomatos'
]

sorted(grocery_list, key=lambda list: list.split()[0].upper())

['Apple', 'Lettuce', 'Tomatos', 'Turkey']

In [17]:
#7.7 Capturing Variable in Anonymous Functions

x = 20
a = lambda y, x=x: x+y
a(10)

30

In [20]:
x=25
b = lambda y, x=x: x+y
b(30)

55

In [21]:
funcs = [lambda x, n=n: x+n for n in range(10)]
for f in funcs:
    print(f(1)) #start from 1


1
2
3
4
5
6
7
8
9
10


In [27]:
#7.8 Making an N-Argument Callable Work As a Callable with Fewer Arguments

from functools import partial

def bar(w, x, y, z):
    print(w, x, y, z)
    
s1 = partial(bar, 66) #fix argument value w=66
s1(2, 3, 4)

66 2 3 4


In [25]:
s1(4, 5, 6)

66 4 5 6


In [28]:
s2 = partial(bar, z=99)
s2(1, 2, 3)

1 2 3 99


In [30]:
s3 = partial(bar, 3, 6, z=22)
s3(5)

3 6 5 22


In [ ]:
#7.9 Replacing Single Method Classes with Functions

from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template
    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))
    


In [31]:
#7.10 Carrying Extra State with Callback Functions

def apply_async(func, args, *, callback): #example of asynchronous processing
    result = func(*args) #compute the result
    callback(result) #Invoke the callback with the result
    
def print_result(result):
    print('Generated:', result)
    
def multuply(x, y):
    return x * y

apply_async(multiply, (7, 11), callback=print_result)

Generated: 77


In [33]:
def add(x, y):
    return x + y

apply_async(add, ('Python', ' is awesome!'), callback=print_result)

Generated: Python is awesome!


In [34]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
    def handler(self, result):
        self.sequence += 1
        print('Test [{}] Generated: {}'.format(self.sequence, result))
        
outcome = ResultHandler()

apply_async(multiply, (7,11), callback=outcome.handler)

Test [1] Generated: 77


In [35]:
apply_async(add, ('Python', ' is awesome!'), callback=outcome.handler)

Test [2] Generated: Python is awesome!


In [37]:
#7.11 Inlining Callback Functions

from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args
        
def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f= func(*args)
        result_queue = Queue
        result_queue.put(None)
        try:
            a = f.send(result)
            apply_async(a.func, a.args, callback=result_queue.put)
        except StopIteration:
            break
    return wrapper



SyntaxError: 'break' outside loop (3545073517.py, line 21)

In [38]:
def test_1():
    result = yield Async(multiply, (7, 11))
    print(result)
    result = yield Async(add, ('Python is', 'awesome!'))
    print(result)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(result)
    print('See ya later!')
    
test_1()

<generator object test_1 at 0x0000014F6AA95EB0>

In [41]:
# 7.12 Accessing Variable Defined Inside a Closure

def sample():
    number = 0
    
    def function(): #closure function
        print('n=', number)
        
    def get_n(): #accessor methods for 'number'
        return number
    
    def set_n(value):
        nonlocal number
        number=value
        
    #attach as function attributes
    
    function.get_n = get_n
    function.set_n = set_n
    return function

In [43]:
f = sample() #global assignment

In [44]:
f()

n= 0


In [46]:
f.set_n(24)
f()

n= 24


In [47]:
f.get_n()
f()

n= 24
